In [1]:
%load_ext autoreload
%autoreload 2

import bz2
import json
import re
import random
import sys
import os
import bz2
import time
import pandas as pd
import numpy as np
from functools import partial
import seaborn as sns
from helpers import *

# PREPROCESSING

In [2]:
PATTERN_INPUT = "../quotebank/quotes-{}.json.bz2"

### Get significant quote count for each speaker

In [3]:
with open("data/local_only/signi-quote-count-combined_1636658426963.json", "r") as f:
    sorted_combined_signi_dict = json.load(f)

In [4]:
signi_list = list(sorted_combined_signi_dict.keys())

### Get the 1000 most quoted speakers

In [5]:
top1000 = signi_list[0:1000]

### Get the quotes

In [6]:
# Define partial function check_if_dem_or_rep_quote using function check_if_party_member_quote
check_if_top_1000_speaker_quote = partial(check_if_speaker_quote, speakers=top1000)

Create files for every year, each file contains a dictionary where the key is the QID of the speaker, and the value is the list of significant quotes attributed to the speaker.
<br><br>
<font color='red'>WARNING: LONG EXECUTION!</font>

In [ ]:
years = [2015, 2016, 2017, 2018, 2019, 2020]

output_list = []

for year in years:
    path_to_input = PATTERN_INPUT.format(year)
    
    # Process quote file
    output_name = process_compressed_json_file(path_to_input, 'data/local_only/top-1000-quotes', year, check_if_top_1000_speaker_quote)
    
    output_list.append(output_name)

print('\n\nOutput file names:')
for file_name in output_list:
    print(file_name)

### Get the wikidata

We used the https://query.wikidata.org/ website to get the relevant wikidata. The SPARQL query is in the following cell.

In [ ]:
SELECT DISTINCT ?item ?itemLabel ?genderLabel ?citizenshipLabel ?languageLabel ?religionLabel ?ethnicLabel ?degreeLabel ?dateOfBirth ?placeOfBirthLabel
WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    VALUES ?item { wd:Q22686 wd:Q1058 wd:Q76 wd:Q450675 wd:Q83106 wd:Q79983 wd:Q43723 wd:Q457786 wd:Q6294 wd:Q359442 wd:Q3629480 wd:Q10218 wd:Q164038 wd:Q1286476 wd:Q7747 wd:Q317521 wd:Q180589 wd:Q39259 wd:Q170581 wd:Q11673 wd:Q291169 wd:Q19946588 wd:Q264766 wd:Q473239 wd:Q213854 wd:Q3099714 wd:Q380900 wd:Q155164 wd:Q6279 wd:Q927550 wd:Q22212 wd:Q4911497 wd:Q434706 wd:Q24313 wd:Q465041 wd:Q313000 wd:Q355522 wd:Q36215 wd:Q523630 wd:Q324546 wd:Q2036942 wd:Q1404825 wd:Q6536656 wd:Q567 wd:Q59155 wd:Q1093629 wd:Q15610524 wd:Q181383 wd:Q18621047 wd:Q3052772 wd:Q25999502 wd:Q36159 wd:Q22316 wd:Q1728820 wd:Q1758777 wd:Q19973684 wd:Q4910865 wd:Q361567 wd:Q311440 wd:Q720027 wd:Q1173911 wd:Q3506475 wd:Q6288836 wd:Q10390 wd:Q11894442 wd:Q358443 wd:Q1017017 wd:Q3606816 wd:Q186304 wd:Q1426 wd:Q11668 wd:Q6446228 wd:Q4359688 wd:Q350843 wd:Q467112 wd:Q463557 wd:Q1779055 wd:Q10993 wd:Q660297 wd:Q2376327 wd:Q1835 wd:Q18164413 wd:Q23303083 wd:Q5229686 wd:Q63879 wd:Q70894 wd:Q15031 wd:Q1186457 wd:Q90005 wd:Q7173106 wd:Q144048 wd:Q973577 wd:Q5300814 wd:Q331401 wd:Q334155 wd:Q439729 wd:Q11459 wd:Q3630922 wd:Q212886 wd:Q27829251 wd:Q3764667 wd:Q1933317 wd:Q2746965 wd:Q27986907 wd:Q16728896 wd:Q7199798 wd:Q921030 wd:Q348577 wd:Q13502385 wd:Q196070 wd:Q69319 wd:Q295610 wd:Q380613 wd:Q318471 wd:Q816459 wd:Q16186315 wd:Q167607 wd:Q207431 wd:Q5605224 wd:Q10125 wd:Q1972377 wd:Q629973 wd:Q2557424 wd:Q52183 wd:Q7437229 wd:Q57675 wd:Q14950689 wd:Q20793563 wd:Q6166666 wd:Q57643 wd:Q4683097 wd:Q3504520 wd:Q2412102 wd:Q348144 wd:Q42311 wd:Q2924590 wd:Q586449 wd:Q5207523 wd:Q2923384 wd:Q6849362 wd:Q646123 wd:Q5812 wd:Q3852721 wd:Q26876 wd:Q10132 wd:Q19953474 wd:Q1773628 wd:Q1077594 wd:Q2023708 wd:Q19843097 wd:Q6386365 wd:Q10853588 wd:Q1253 wd:Q57661 wd:Q718601 wd:Q7238178 wd:Q719568 wd:Q18388174 wd:Q862439 wd:Q221997 wd:Q1652493 wd:Q58217 wd:Q316681 wd:Q4661862 wd:Q3243189 wd:Q41533 wd:Q595978 wd:Q4861362 wd:Q1135767 wd:Q27612210 wd:Q47213 wd:Q4768021 wd:Q15935 wd:Q5216414 wd:Q928118 wd:Q1097422 wd:Q28229066 wd:Q707164 wd:Q24248265 wd:Q333533 wd:Q2622812 wd:Q371118 wd:Q529294 wd:Q24676423 wd:Q557758 wd:Q122304 wd:Q1516578 wd:Q331904 wd:Q10599 wd:Q152451 wd:Q134068 wd:Q10296812 wd:Q6414956 wd:Q57665 wd:Q132345 wd:Q3001998 wd:Q6173283 wd:Q6809265 wd:Q630111 wd:Q461391 wd:Q296036 wd:Q3518013 wd:Q331719 wd:Q2444013 wd:Q7356366 wd:Q1670096 wd:Q3787422 wd:Q2414784 wd:Q946 wd:Q2979333 wd:Q1138109 wd:Q40063430 wd:Q359118 wd:Q7173778 wd:Q352159 wd:Q4695503 wd:Q24698460 wd:Q26837187 wd:Q298443 wd:Q608839 wd:Q7839350 wd:Q173472 wd:Q2287947 wd:Q82816 wd:Q3313434 wd:Q14950026 wd:Q55223040 wd:Q10379941 wd:Q434858 wd:Q1176671 wd:Q1235731 wd:Q459830 wd:Q6288876 wd:Q352093 wd:Q22237 wd:Q260042 wd:Q35489166 wd:Q7291655 wd:Q348857 wd:Q3308102 wd:Q16196878 wd:Q478161 wd:Q529344 wd:Q5694014 wd:Q1280022 wd:Q3180174 wd:Q660606 wd:Q4679094 wd:Q27074185 wd:Q29033668 wd:Q7825680 wd:Q22279 wd:Q1529538 wd:Q5336135 wd:Q417657 wd:Q334388 wd:Q4046340 wd:Q1701249 wd:Q28051620 wd:Q727798 wd:Q2926405 wd:Q3337690 wd:Q6451434 wd:Q3956796 wd:Q1557544 wd:Q10514 wd:Q1494363 wd:Q359888 wd:Q212648 wd:Q3522251 wd:Q16090838 wd:Q16732034 wd:Q14949474 wd:Q1189011 wd:Q15222407 wd:Q11310708 wd:Q3595385 wd:Q1174175 wd:Q1391309 wd:Q5386557 wd:Q3304418 wd:Q1119051 wd:Q192533 wd:Q230733 wd:Q334541 wd:Q5108471 wd:Q573246 wd:Q515161 wd:Q3528364 wd:Q3013178 wd:Q1100710 wd:Q17811880 wd:Q2745305 wd:Q720121 wd:Q20812953 wd:Q5620734 wd:Q5593952 wd:Q11301335 wd:Q432473 wd:Q192635 wd:Q16136520 wd:Q991551 wd:Q2614593 wd:Q1741083 wd:Q440324 wd:Q1396178 wd:Q456730 wd:Q11836360 wd:Q5566385 wd:Q22222 wd:Q6221385 wd:Q934781 wd:Q1231482 wd:Q7441226 wd:Q203781 wd:Q16499788 wd:Q21622845 wd:Q1124 wd:Q434804 wd:Q11963798 wd:Q16196017 wd:Q14951440 wd:Q6182718 wd:Q532423 wd:Q1051086 wd:Q7531710 wd:Q5162259 wd:Q312556 wd:Q484605 wd:Q354529 wd:Q222789 wd:Q3105215 wd:Q1991889 wd:Q162629 wd:Q263725 wd:Q3992490 wd:Q381880 wd:Q19848 wd:Q8000233 wd:Q429539 wd:Q7441250 wd:Q2239218 wd:Q57446 wd:Q16196005 wd:Q1173842 wd:Q267902 wd:Q174614 wd:Q1181130 wd:Q5269370 wd:Q57783 wd:Q15972549 wd:Q1102021 wd:Q83287 wd:Q15064277 wd:Q157 wd:Q6767270 wd:Q2631300 wd:Q34086 wd:Q3741557 wd:Q15126384 wd:Q929581 wd:Q333937 wd:Q152316 wd:Q215057 wd:Q438465 wd:Q437993 wd:Q15637781 wd:Q294460 wd:Q12326156 wd:Q3090307 wd:Q158813 wd:Q743162 wd:Q25136935 wd:Q7426870 wd:Q4965336 wd:Q6196807 wd:Q3633236 wd:Q607 wd:Q11571 wd:Q27923598 wd:Q297252 wd:Q16106910 wd:Q72245 wd:Q16106332 wd:Q318503 wd:Q3518911 wd:Q1173155 wd:Q15051763 wd:Q381157 wd:Q3162959 wd:Q18527503 wd:Q981496 wd:Q21064126 wd:Q10502635 wd:Q58077 wd:Q10450056 wd:Q57657 wd:Q525362 wd:Q7518921 wd:Q4571225 wd:Q7279347 wd:Q151892 wd:Q4679786 wd:Q15647498 wd:Q5111202 wd:Q55800 wd:Q3897557 wd:Q284169 wd:Q235068 wd:Q58132 wd:Q5605742 wd:Q16145291 wd:Q11153 wd:Q2271796 wd:Q239411 wd:Q455854 wd:Q6415427 wd:Q15932545 wd:Q6336063 wd:Q13133 wd:Q12892062 wd:Q3561087 wd:Q39318 wd:Q42493 wd:Q3376386 wd:Q348618 wd:Q950413 wd:Q2605384 wd:Q194419 wd:Q1770624 wd:Q4384608 wd:Q434909 wd:Q3503829 wd:Q7914038 wd:Q16104403 wd:Q16728747 wd:Q339256 wd:Q1177200 wd:Q16885910 wd:Q319072 wd:Q5284 wd:Q505200 wd:Q14536 wd:Q16845347 wd:Q365882 wd:Q7331672 wd:Q1173399 wd:Q24005532 wd:Q19953601 wd:Q135209 wd:Q182788 wd:Q357994 wd:Q625178 wd:Q3414516 wd:Q1428440 wd:Q4235 wd:Q3135561 wd:Q5335247 wd:Q15477554 wd:Q10515624 wd:Q351004 wd:Q11894443 wd:Q730063 wd:Q25369 wd:Q652815 wd:Q3595131 wd:Q14950467 wd:Q4025936 wd:Q1680776 wd:Q6885827 wd:Q3634346 wd:Q4792148 wd:Q1928572 wd:Q538868 wd:Q520173 wd:Q16214685 wd:Q19730927 wd:Q183058 wd:Q419976 wd:Q195616 wd:Q195725 wd:Q6789115 wd:Q10381179 wd:Q19825841 wd:Q13427287 wd:Q16150230 wd:Q561487 wd:Q25756209 wd:Q41173 wd:Q373491 wd:Q1077115 wd:Q1077119 wd:Q8052224 wd:Q16146870 wd:Q755831 wd:Q40715 wd:Q15820929 wd:Q1445246 wd:Q55584121 wd:Q162202 wd:Q983163 wd:Q14330096 wd:Q312015 wd:Q186215 wd:Q16207227 wd:Q3196830 wd:Q57336 wd:Q10430120 wd:Q16093851 wd:Q4492815 wd:Q1382365 wd:Q2447626 wd:Q1363468 wd:Q479171 wd:Q1280288 wd:Q233748 wd:Q319591 wd:Q1055449 wd:Q888132 wd:Q7384672 wd:Q19873951 wd:Q33190271 wd:Q360491 wd:Q767499 wd:Q8002987 wd:Q3337844 wd:Q20118419 wd:Q19650494 wd:Q31683 wd:Q472241 wd:Q19665666 wd:Q268386 wd:Q25997639 wd:Q7287916 wd:Q1077381 wd:Q28066064 wd:Q22360 wd:Q381477 wd:Q15991542 wd:Q335552 wd:Q335697 wd:Q1882744 wd:Q16981006 wd:Q43370301 wd:Q10456471 wd:Q9545 wd:Q1340803 wd:Q2435968 wd:Q17183175 wd:Q3339779 wd:Q922169 wd:Q318204 wd:Q1279758 wd:Q608639 wd:Q3018779 wd:Q37084447 wd:Q15072754 wd:Q10738 wd:Q1855840 wd:Q16195604 wd:Q17490263 wd:Q124853 wd:Q6761357 wd:Q16208073 wd:Q1387131 wd:Q234653 wd:Q6173366 wd:Q335680 wd:Q3018962 wd:Q3210225 wd:Q7793121 wd:Q138745 wd:Q17019633 wd:Q9488 wd:Q4756662 wd:Q6769423 wd:Q57788 wd:Q11339601 wd:Q22260 wd:Q1027519 wd:Q1206954 wd:Q3532495 wd:Q16197112 wd:Q4931714 wd:Q302581 wd:Q5115650 wd:Q1981766 wd:Q468442 wd:Q562522 wd:Q2924879 wd:Q926069 wd:Q19217 wd:Q1177029 wd:Q268584 wd:Q1560250 wd:Q7437083 wd:Q5489177 wd:Q471883 wd:Q15840165 wd:Q373064 wd:Q1176177 wd:Q3518502 wd:Q25351017 wd:Q270820 wd:Q5300433 wd:Q1928041 wd:Q4716651 wd:Q22639331 wd:Q263802 wd:Q470774 wd:Q173789 wd:Q4898286 wd:Q15047261 wd:Q7518284 wd:Q731502 wd:Q5765114 wd:Q365144 wd:Q702233 wd:Q1378237 wd:Q557730 wd:Q1190856 wd:Q7418472 wd:Q1378351 wd:Q4915447 wd:Q1185730 wd:Q5228891 wd:Q246724 wd:Q2575239 wd:Q383983 wd:Q7182389 wd:Q615 wd:Q3720084 wd:Q2412509 wd:Q4986999 wd:Q4886139 wd:Q1335436 wd:Q1341137 wd:Q325374 wd:Q16732648 wd:Q17414660 wd:Q1000592 wd:Q11040742 wd:Q266647 wd:Q483325 wd:Q50678 wd:Q920403 wd:Q1271675 wd:Q16105043 wd:Q21463799 wd:Q1174985 wd:Q948505 wd:Q539493 wd:Q266613 wd:Q466220 wd:Q1515752 wd:Q970501 wd:Q256380 wd:Q887111 wd:Q2846724 wd:Q14945660 wd:Q207 wd:Q359298 wd:Q430518 wd:Q15123969 wd:Q5375903 wd:Q128898 wd:Q3038277 wd:Q124138 wd:Q1303383 wd:Q580808 wd:Q2618003 wd:Q1740168 wd:Q483417 wd:Q16194481 wd:Q7804091 wd:Q4496 wd:Q461727 wd:Q1189 wd:Q58198 wd:Q5280837 wd:Q1137104 wd:Q4749380 wd:Q24678643 wd:Q12470060 wd:Q3786221 wd:Q16213945 wd:Q16233613 wd:Q75820 wd:Q6409269 wd:Q16724461 wd:Q6162716 wd:Q4662015 wd:Q3517911 wd:Q970720 wd:Q1057736 wd:Q1112289 wd:Q269909 wd:Q16123033 wd:Q16205086 wd:Q5570448 wd:Q3313521 wd:Q1077015 wd:Q923242 wd:Q7288518 wd:Q5002395 wd:Q16683338 wd:Q521648 wd:Q14948763 wd:Q59315 wd:Q304251 wd:Q516515 wd:Q1813692 wd:Q3807514 wd:Q4962244 wd:Q7546539 wd:Q3530322 wd:Q1757345 wd:Q24005613 wd:Q1388440 wd:Q934898 wd:Q28086102 wd:Q22004534 wd:Q3601035 wd:Q978276 wd:Q4961919 wd:Q16192221 wd:Q510523 wd:Q295492 wd:Q1174678 wd:Q1251779 wd:Q7846832 wd:Q108447 wd:Q16204402 wd:Q1398542 wd:Q1357640 wd:Q902879 wd:Q24004546 wd:Q1049213 wd:Q244338 wd:Q483507 wd:Q13475242 wd:Q214204 wd:Q47447 wd:Q371327 wd:Q19561555 wd:Q5214072 wd:Q16751186 wd:Q4978124 wd:Q21466514 wd:Q1077730 wd:Q32620 wd:Q21611418 wd:Q16300641 wd:Q13909 wd:Q11702 wd:Q18159975 wd:Q1399415 wd:Q717959 wd:Q1444383 wd:Q20684288 wd:Q94831 wd:Q160902 wd:Q713383 wd:Q6847094 wd:Q21622916 wd:Q2737207 wd:Q57641 wd:Q14950971 wd:Q7423486 wd:Q17386283 wd:Q4772976 wd:Q16150045 wd:Q17163472 wd:Q5221979 wd:Q28972111 wd:Q14647783 wd:Q143180 wd:Q1395713 wd:Q6415053 wd:Q3758515 wd:Q2679005 wd:Q5107742 wd:Q1175688 wd:Q131112 wd:Q15712142 wd:Q38952 wd:Q4910369 wd:Q5026232 wd:Q1175128 wd:Q14950655 wd:Q12121376 wd:Q23685 wd:Q337453 wd:Q16204680 wd:Q1060609 wd:Q1282411 wd:Q206 wd:Q5214486 wd:Q192682 wd:Q1091242 wd:Q16202391 wd:Q38858352 wd:Q486359 wd:Q30600781 wd:Q723295 wd:Q6733078 wd:Q304095 wd:Q19946119 wd:Q2014922 wd:Q173839 wd:Q887766 wd:Q530705 wd:Q45940 wd:Q6847862 wd:Q11320682 wd:Q16229793 wd:Q30981158 wd:Q431248 wd:Q467414 wd:Q56486914 wd:Q152871 wd:Q621818 wd:Q5714209 wd:Q9282722 wd:Q5607720 wd:Q1822266 wd:Q16238905 wd:Q574980 wd:Q2907612 wd:Q1857141 wd:Q44857 wd:Q47388938 wd:Q16236542 wd:Q6194498 wd:Q210447 wd:Q16197018 wd:Q1149781 wd:Q12068555 wd:Q3531258 wd:Q163225 wd:Q500551 wd:Q12967201 wd:Q586579 wd:Q16229842 wd:Q288359 wd:Q27927683 wd:Q16727692 wd:Q2021513 wd:Q138121 wd:Q6504777 wd:Q230156 wd:Q1152925 wd:Q862186 wd:Q16196728 wd:Q16185439 wd:Q708694 wd:Q5279689 wd:Q513091 wd:Q20235524 wd:Q143572 wd:Q43380465 wd:Q3441393 wd:Q3530233 wd:Q1683881 wd:Q1368597 wd:Q1179916 wd:Q12720234 wd:Q19661212 wd:Q503706 wd:Q3011889 wd:Q189490 wd:Q6789172 wd:Q58311 wd:Q6245218 wd:Q311007 wd:Q881188 wd:Q21001 wd:Q2018023 wd:Q5225020 wd:Q4707000 wd:Q10500229 wd:Q16235485 wd:Q26132883 wd:Q16189318 wd:Q776784 wd:Q6933491 wd:Q16292920 wd:Q1356377 wd:Q1795070 wd:Q258825 wd:Q42574 wd:Q7306657 wd:Q1543207 wd:Q1285979 wd:Q18159074 wd:Q16214242 wd:Q971308 wd:Q14913044 wd:Q16019429 wd:Q293260 wd:Q16205594 wd:Q3017153 wd:Q6383803 wd:Q1148728 wd:Q10524 wd:Q573463 wd:Q209479 wd:Q1077303 wd:Q381475 wd:Q1619538 wd:Q544464 wd:Q9557 wd:Q11116 wd:Q16194501 wd:Q16187041 wd:Q4054762 wd:Q6388290 wd:Q16194857 wd:Q4291295 wd:Q655759 wd:Q25183273 wd:Q1193089 wd:Q3501742 wd:Q242201 wd:Q11169832 wd:Q15854233 wd:Q3520790 wd:Q13639814 wd:Q8064960 wd:Q685723 wd:Q16213987 wd:Q16209157 wd:Q2834185 wd:Q178882 wd:Q1827902 wd:Q19519946 wd:Q244312 wd:Q159178 wd:Q21213245 wd:Q1179451 wd:Q16149064 wd:Q1391397 wd:Q16212482 wd:Q471871 wd:Q18336221 wd:Q16732613 wd:Q18702995 wd:Q188137 wd:Q2663804 wd:Q2588612 wd:Q6103950 wd:Q455979 wd:Q16145764 wd:Q431216 wd:Q7331434 wd:Q4806029 wd:Q6176439 wd:Q6749904 wd:Q6774889 wd:Q18157918 wd:Q27840142 wd:Q6968942 wd:Q3041278 wd:Q1112016 wd:Q3017201 wd:Q1175520 wd:Q60772 wd:Q1449940 wd:Q14755515 wd:Q1176654 wd:Q21175197 wd:Q7152168 wd:Q2599 wd:Q18111 wd:Q15734070 wd:Q6220673 wd:Q44063 wd:Q6751197 wd:Q993472 wd:Q5525857 wd:Q1135774 wd:Q299574 wd:Q1726811 wd:Q16243986 wd:Q332528 wd:Q483309 wd:Q5110039 wd:Q16255546 wd:Q16215583 wd:Q11339500 wd:Q7293098 wd:Q7417125 wd:Q3345004 wd:Q1060949 wd:Q235132 wd:Q4965230 wd:Q11693085 wd:Q1179039 wd:Q3317069 wd:Q15280 }
    
    OPTIONAL { ?item wdt:P21 ?gender. }
    OPTIONAL { ?item wdt:P27 ?citizenship. }
    OPTIONAL { ?item wdt:P569 ?dateOfBirth. }
    OPTIONAL { ?item wdt:P103 ?language. }
    OPTIONAL { ?item wdt:P140 ?religion. }
    OPTIONAL { ?item wdt:P172 ?ethnic. }
    OPTIONAL { ?item wdt:P512 ?degree. }
  }
}

In [9]:
with open("data/local_only/top-1000-wikidata-raw-with-occupation.json", "r", encoding='utf-8') as f:
    top1000_wiki = json.load(f)

In [10]:
top1000_wiki_merged = merge_wikidata_entries(
    top1000_wiki,
    ['itemLabel', 'genderLabel', 'citizenshipLabel', 'religionLabel', 'ethnicLabel', 'degreeLabel', 'dateOfBirth', 'languageLabel']
)

In [11]:
write_json_to_file('data/local_only/top-1000-wikidata-occupation-merged', top1000_wiki_merged)

'data/local_only/top-1000-wikidata-occupation-merged_1639762772759.json'

### Combine the quotes

In [12]:
top1000_list = list(top1000_wiki_merged.values())

In [13]:
quotes_files = [
    "data/local_only/top-1000-quotes-2015_1638929459545.json",
    "data/local_only/top-1000-quotes-2016_1638931024701.json",
    "data/local_only/top-1000-quotes-2017_1638934319108.json",
    "data/local_only/top-1000-quotes-2018_1638935809502.json",
    "data/local_only/top-1000-quotes-2019_1638936901748.json",
    "data/local_only/top-1000-quotes-2020_1638937168661.json"
]

quotes_combined = combine_quotes_files_2nd_format(top1000_list, quotes_files)

In [14]:
write_json_to_file('data/local_only/top-1000-quotes-combined', quotes_combined)

'data/local_only/top-1000-quotes-combined_1639762781876.json'

### Filter the quotes

Some of the quotes in the database do not represent actual quotes, but instead contain junk like html tags, source code, or text from the webpage where the source article was published.
<br>
We filter these quotes out so our dataset is not polluted by junk data. We have found a few filters which detect most of the junk data, while maintaining a low false positive rate:
<ul>
    <li>quotes which contains very long 'words' - more than 50 characters</li>
    <li>quotes which contain URLs - these usually contain other junk characters</li>
    <li>quotes which contains JSON-like key-value pairs</li>
    <li>quotes which contain a lot of special characters (more than 10% of total characters)</li>
</ul>

In [15]:
quotes_filtered = quotes_combined.copy()

In [16]:
quotes_filtered, dirty_quotes = filter_quotes_2nd_format(quotes_filtered)

Show some filtered quotes:

In [17]:
for entry in dirty_quotes[0:5]:
    print(entry['quotation'], end='\n\n')

DPM Teo Chee Hean (who is Acting PM as I am away) has advised the President to issue the Writ of Election for Bukit Batok constituency. Nomination Day will be 27 April (Wednesday) and Polling Day 7 May (Saturday). Details here: http://bit.ly/23ZgJUj DPM Teo and I spoke earlier. Now that the Budget is out of the way, we are calling this by-election to enable residents of Bukit Batok to have their own MP as we work together to improve Singaporeans' lives. Partnership between Government and citizens is key to our success. I hope Bukit Batok residents will elect the candidate who will be concerned for their welfare, represent them ably in Parliament, and make Bukit Batok the best home for them.

a strong sig & shy; nal of the U.S. com & shy; mit & shy; ment to con & shy; tin & shy; ue its deep en & shy; gage & shy; ment in the re & shy; gion.

sosur bari & Acirc;

Thanks to every mother-daughter-sister-friend-cousin-neice-companion-partner who joined us on the #Girl's GetAwayCruise. #Omago

### Concatenate the quotes

In [18]:
quotes_concat = quotes_filtered.copy()

In [19]:
quotes_concat = concatenate_quotes_2nd_format(quotes_concat)

In [20]:
write_json_to_file('data/local_only/top-1000-quotes-concatenated', quotes_concat)

'data/local_only/top-1000-quotes-concatenated_1639762876567.json'

### LIWC

In [21]:
import bz2
import json
import pandas as pd
import csv

In [22]:
# Load the concatenated quotes for top 100 politician
with open('data/local_only/top-1000-quotes-concatenated_1638964313725.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

After getting the data, we extract the quote ID and the concatenated quote of each politician, and write them to `input_data1.csv` for the LIWC personality analysis.

In [23]:
with open('data/local_only/top_1000_input_data_1.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["qid", "quote"])
    for qid, all_value in data.items():
        quote = all_value["quotations"]
        writer.writerow([qid, quote])

In [24]:
liwc = pd.read_csv('data/local_only/top_1000_output_1.csv')

In [25]:
# Visualise a random sample
liwc.sample()

,Source (A),Source (B),WC,WPS,Sixltr,Dic,Pronoun,I,We,Self,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
706,Q9282722,It's been a great weekend for Romain. Not the...,978,11.78,11.76,78.83,12.99,1.43,5.42,6.85,...,3.89,0.0,0.0,0.0,0.0,0.92,0.0,3.68,1.02,0.0
